In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import ultraplot as uplt

This is an example notebook for AVIRIS radiance data. The dataset is available at: https://aviris.jpl.nasa.gov/dataportal/.

In [ ]:
# The datasets are in the form of hierarchically-nested netCDF files, which cannot be opened as a standard xarray dataset.
# The files can only be opened as a datatree structure, which is only available in newer versions of xarray.
path = 'datasets\AV320230710t193100_016_L1B_RDN_cbeae6f8_RDN.nc'
ds = xr.open_datatree(path)
ds

In [ ]:
## Retrieve radiance information & assign arbitrary coordinates
samples_coords = np.arange(1234)
lines_coords = np.arange(1280)
radiance = ds.radiance.radiance.assign_coords({'samples':samples_coords, 'lines':lines_coords})
radiance

In [ ]:
radiance.sel(wavelength='766.5', method='nearest').plot()

In [ ]:
# Plot the spectrum of a single pixel
radiance.sel(lines=600, samples=300).plot()

In [ ]:
# Calculate the HDFI index
radiance_2430 = radiance.sel(wavelength='2430', method='nearest')
radiance_2060 = radiance.sel(wavelength='2060', method='nearest')
HDFI = (radiance_2430 - radiance_2060)/(radiance_2430 + radiance_2060)

In [ ]:
radiance_2430.plot()

In [ ]:
radiance_2060.plot()

In [ ]:
HDFI.plot()
# Values are all negative. There are likely no active fires.

Data related to the 2025 Palisades fire can be found at:

https://www.earthdata.nasa.gov/data/alerts-outages/aviris-3-l1b-radiance-data-related-california-fires-now-available

In [ ]:
pal_rad_path = 'datasets\palisades_fire\AV320250111t210400_005_L1B_RDN_3f4aef90_RDN.nc'
pal_mask_path = 'datasets\palisades_fire\AV320250111t210400_005_L1B_RDN_3f4aef90_BANDMASK.nc'
pal_ds = xr.open_datatree(pal_rad_path)
pal_ds

In [ ]:
samples_coords = np.arange(1234)
lines_coords = np.arange(1280)
# Assign dummy coordinates
pal_radiance = pal_ds.radiance.radiance.assign_coords({'samples':samples_coords, 'lines':lines_coords})
pal_radiance

In [ ]:
pal_mask = xr.open_dataset(pal_mask_path).assign_coords({'samples':samples_coords, 'lines':lines_coords})
pal_mask # Band mask file; used for masking out low-quality spectral bands which were interpolated
# The 36 bands cover the full wavelength band of 284; each band value is an 8-bit unsigned integer
# Which in boolean corresponds to which of the 8 wavelengths in each band were masked.

In [ ]:
# Generate an RGB image
def normalize(band):
    band_min = band.min()
    band_max = band.max()
    return (band - band_min) / (band_max - band_min)

red_ = pal_radiance.sel(wavelength=700, method='nearest')
green_ = pal_radiance.sel(wavelength=500, method='nearest')
blue_ = pal_radiance.sel(wavelength=300, method='nearest')

red = normalize(red_)
green = normalize(green_) 
blue = normalize(blue_)
rgb_image = np.dstack((red.values, green.values, blue.values))
plt.figure(figsize=(10, 10))
plt.imshow(rgb_image)

In [ ]:
red_ = pal_radiance.sel(wavelength=2200, method='nearest')
green_ = pal_radiance.sel(wavelength=700, method='nearest')
blue_ = pal_radiance.sel(wavelength=300, method='nearest')

red = normalize(red_)
green = normalize(green_)
blue = normalize(blue_)
rgb_image = np.dstack((red.values, green.values, blue.values))
plt.figure(figsize=(10, 10))
plt.imshow(rgb_image)

In [ ]:
# Masking is a pain so let's visualize the original data
# Calculate the HDFI index
pal_rad_2430 = pal_radiance.sel(wavelength=slice(2420,2440)).mean(dim='wavelength')
pal_rad_2060 = pal_radiance.sel(wavelength=slice(2050,2070)).mean(dim='wavelength')
pal_HFDI = (pal_rad_2430 - pal_rad_2060)/(pal_rad_2430 + pal_rad_2060)

In [ ]:
fig, ax = uplt.subplots(refwidth=6)
pal_HFDI.plot(ax=ax, vmin=-.5, vmax=.5, discrete=False, cmap='RdBu_r')
ax.format(
    yreverse=True,
    suptitle='Palisades Fire 2025-01-11 HFDI Index'
)

In [ ]:
bins = np.linspace(-0.4, 0.4, 500)
fig, ax = uplt.subplots(refwidth=6, refaspect=(3,1))
_ = pal_HDFI.plot.hist(bins=bins, ax=ax)
ax.format(
    suptitle='Distribution of pixel HFDI'
)